<a href="https://colab.research.google.com/github/JhonUsuga/ModelosEntregaFinal/blob/main/03%20-%20modelo%20con%20preprocesado%20Random%20Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

udea_ai_4_eng_20251_pruebas_saber_pro_colombia_path = kagglehub.competition_download('udea-ai-4-eng-20251-pruebas-saber-pro-colombia')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/udea-ai-4-eng-20251-pruebas-saber-pro-colombia/submission_example.csv
/kaggle/input/udea-ai-4-eng-20251-pruebas-saber-pro-colombia/train.csv
/kaggle/input/udea-ai-4-eng-20251-pruebas-saber-pro-colombia/test.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
train = pd.read_csv("/kaggle/input/udea-ai-4-eng-20251-pruebas-saber-pro-colombia/train.csv")
test = pd.read_csv("/kaggle/input/udea-ai-4-eng-20251-pruebas-saber-pro-colombia/test.csv")

In [ ]:
columnas_categoricas_con_nulos = [
    'ESTU_VALORMATRICULAUNIVERSIDAD',
    'ESTU_HORASSEMANATRABAJA',
    'FAMI_ESTRATOVIVIENDA',
    'FAMI_TIENEINTERNET',
    'FAMI_EDUCACIONPADRE',
    'FAMI_TIENELAVADORA',
    'FAMI_TIENEAUTOMOVIL',
    'ESTU_PAGOMATRICULAPROPIO',
    'FAMI_TIENECOMPUTADOR',
    'FAMI_TIENEINTERNET.1',
    'FAMI_EDUCACIONMADRE'
]

# Imputar con el modo
for col in columnas_categoricas_con_nulos:
    train[col].fillna(train[col].mode()[0], inplace=True)
    test[col].fillna(train[col].mode()[0], inplace=True)  # usar modo del train

columnas_categoricas = [
    'ESTU_PRGM_ACADEMICO',
    'ESTU_PRGM_DEPARTAMENTO',
    'ESTU_VALORMATRICULAUNIVERSIDAD',
    'ESTU_HORASSEMANATRABAJA',
    'FAMI_ESTRATOVIVIENDA',
    'FAMI_TIENEINTERNET',
    'FAMI_EDUCACIONPADRE',
    'FAMI_TIENELAVADORA',
    'FAMI_TIENEAUTOMOVIL',
    'ESTU_PRIVADO_LIBERTAD',
    'ESTU_PAGOMATRICULAPROPIO',
    'FAMI_TIENECOMPUTADOR',
    'FAMI_TIENEINTERNET.1',
    'FAMI_EDUCACIONMADRE'
]

# One-hot encoding
train = pd.get_dummies(train, columns=columnas_categoricas, drop_first=True)
test = pd.get_dummies(test, columns=columnas_categoricas, drop_first=True)

# Alinear columnas entre train y test
X = train.drop(columns=["RENDIMIENTO_GLOBAL", "ID"])
y = train["RENDIMIENTO_GLOBAL"].map({'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3})
test_ids = test["ID"]

test = test.reindex(columns=X.columns, fill_value=0)

# Normalización de coeficientes
columnas_coef = ['coef_1', 'coef_2', 'coef_3', 'coef_4']
scaler = MinMaxScaler()
X[columnas_coef] = scaler.fit_transform(X[columnas_coef])
test[columnas_coef] = scaler.transform(test[columnas_coef])

/tmp/ipykernel_35/349594100.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(train[col].mode()[0], inplace=True)
/tmp/ipykernel_35/349594100.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_val)
print("Accuracy en validación:", accuracy_score(y_val, y_pred))

Accuracy en validación: 0.40650541516245486


In [ ]:
pred = modelo.predict(test)

# Volver a etiquetas
map_inv = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
pred_etiquetas = [map_inv[i] for i in pred]

# Guardar submission
submission = pd.DataFrame({
    "ID": test_ids,
    "RENDIMIENTO_GLOBAL": pred_etiquetas
})
submission.to_csv("/kaggle/working/submission.csv", index=False)
print("submission.csv generado")

submission.csv generado
